Note: Google collab doesn't support OpenCV. So please open it in Jupyter notebook or the local machine instead 😀.

#Install modules


In [ ]:
!pip install -r requirements.txt

#Import modules

In [1]:
import cv2
import os
import numpy as np
import keras
import matplotlib.pyplot as plt
from tensorflow.keras.applications import VGG16
from keras import backend as K
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Activation, LSTM
import h5py

#Initial variables



In [2]:
DIR_TR = "dataset/Train/"
DIR_TE = "dataset/Test/"
DIR_VA = "dataset/Valid/"

# Frame size  
IMG_SIZE = 224
IMG_SIZE_TUPLE = (IMG_SIZE, IMG_SIZE)

# Number of channels (RGB)
NUM_CHANNELS = 3
# Flat frame size
IMG_SIZE_FLAT = IMG_SIZE**2 * NUM_CHANNELS

# Number of classes for classification (Violence-No Violence)
NUM_CLASSES = 2
CLASSES_LIST = ['NonViolence', 'Violence']
assert len(CLASSES_LIST) == NUM_CLASSES

SEQUENCE_LENGTH = 20

#VGG16 Model

In [3]:
image_model = VGG16(include_top=True, weights='imagenet')
image_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [4]:
transfer_layer = image_model.get_layer('fc2')
image_model_transfer = Model(inputs=image_model.input,
                             outputs=transfer_layer.output)

#Frame extraction function

## Function

Get array of each frame

In [14]:

def get_frames(vid_path):
    cap = cv2.VideoCapture(vid_path)
    
    new_seq = SEQUENCE_LENGTH
    
    total_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    split_size = int(min(3, max(total_frames // new_seq, 1)))
    chunk_size = total_frames / split_size
    skip_frame = max(1, chunk_size // new_seq)
    
    for c in range(split_size):
        initial_frame = new_seq * c * skip_frame
        
        frames_list = []
        for i in range(new_seq):
            cap.set(cv2.CAP_PROP_POS_FRAMES, (i * skip_frame) + initial_frame)
            ret, frame = cap.read()
            if not ret:
                break
            
            RGB_img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                
            res = cv2.resize(RGB_img, 
                             dsize=IMG_SIZE_TUPLE,
                             interpolation=cv2.INTER_CUBIC)
            
            frames_list.append(res)

            
        if len(frames_list) != SEQUENCE_LENGTH:
            continue
        
        result = np.array(frames_list)
        result = (result / 255).astype(np.float16)
        
        yield result

Extract features each frame from video

In [15]:
def extracted_features(vid_path):
    gen = get_frames(vid_path)
    for chunk in gen:
        shape = (SEQUENCE_LENGTH, 4096)
        transfer_values = np.zeros(shape=shape, dtype=np.float16)
        transfer_values = image_model_transfer.predict(chunk)
        
        yield transfer_values
    

Create dataset

In [16]:
def create_dataset(dataset_dir):
    class_dataset_dir = os.listdir(dataset_dir)
    features = []
    labels = []
    
    
    for dir in class_dataset_dir:
        videos_list = os.listdir(os.path.join(dataset_dir, dir))
        count = 0
        for vid in videos_list:
            gen_path = os.path.join(dataset_dir, dir, vid)
            gen = extracted_features(gen_path)
            for chunk in gen:
                features.append(chunk)
                if dir == CLASSES_LIST[0]:
                    labels.append([1, 0])
                elif dir == CLASSES_LIST[1]:
                    labels.append([0, 1])
            count += 1
            
            print(dir, '{:.2f}%'.format((count /len(videos_list))*100))
    return features, labels

##Create dataset for video

Note: Please skip this section if you already save the dataset.

In [17]:
features_tr, labels_tr = create_dataset(DIR_TR)
features_te, labels_te = create_dataset(DIR_TE)
features_va, labels_va = create_dataset(DIR_VA)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'dataset/Train/'

##Write and load dataset

After you finish the create dataset section. Uncomment this below code to save the dataset and when you've done, Comment it again to make sure you won't overwrite it.

In [ ]:
# with h5py.File('features_labels/datav6.h5', mode='w') as file:
#     file.create_dataset('X_tr', data=features_tr)
#     file.create_dataset('X_te', data=features_te)
#     file.create_dataset('X_va', data=features_va)
    
#     file.create_dataset('y_tr', data=labels_tr)
#     file.create_dataset('y_te', data=labels_te)
#     file.create_dataset('y_va', data=labels_va)

Load the dataset from hdf5 file

In [ ]:
with h5py.File('features_labels/datav6.h5', mode='r') as file:
    X_tr = file['X_tr'][:]
    X_te = file['X_te'][:]
    X_va = file['X_va'][:]
    
    y_tr = file['y_tr'][:]
    y_te = file['y_te'][:]
    y_va = file['y_va'][:]

##Check data

In [ ]:
print("Train features", len(X_tr))
print("Test features", len(X_te))
print("Validation features", len(X_va))

In [ ]:
print("Train label", len(y_tr))
print("Test label", len(y_te))
print("Validation label", len(y_va))

#LSTM model

##Create model


In [ ]:
chunk_size = 4096
n_chunks = SEQUENCE_LENGTH
rnn_size = 512

model = Sequential()
model.add(LSTM(rnn_size, input_shape=(n_chunks, chunk_size)))
model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dense(50))
model.add(Activation('sigmoid'))
model.add(Dense(2))
model.add(Activation('softmax'))


##Train model

In [ ]:
from keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
epoch = 200
batchS = 500

early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 15, mode = 'min', restore_best_weights = True)

opt = Adam(learning_rate=0.001)
model.compile(loss='mean_squared_error', optimizer=opt, metrics=['accuracy'])
history = model.fit(x=np.array(X_tr), y=np.array(y_tr), epochs=epoch,
                    validation_data=(np.array(X_va), np.array(y_va)),
                    shuffle=True,
                    batch_size=batchS, verbose=2,
                    callbacks=[early_stopping_callback])

##Evaluate model

In [ ]:
result = model.evaluate(np.array(X_te), np.array(y_te))

In [ ]:
for name, value in zip(model.metrics_names, result):
    print(name, value)

##Plot graph.

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.savefig('destination_path.eps', format='eps', dpi=1000)
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.savefig('destination_path1.eps', format='eps', dpi=1000)
plt.show()


##Save model and save weights.

In [ ]:
model.save('model/vggLSTM/modelv4_2.h5')
model.save_weights('model/vggLSTM/model_weightsv4_2.h5')